In [1]:
# from langchain.document_loaders import DirectoryLoader
from langchain_community.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain.document_loaders.pdf import PyPDFDirectoryLoader
# from langchain.embeddings import OpenAIEmbeddings
#from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
#import openai 
from dotenv import load_dotenv
import os
import shutil


# from langchain.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langchain.docstore.document import Document

In [2]:
# Load environment variables. Assumes that project contains .env file with API keys
# load_dotenv()
#---- Set OpenAI API key 
# Change environment variable name from "OPENAI_API_KEY" to the name given in 
# your .env file.
#openai.api_key = os.environ['OPENAI_API_KEY']

CHROMA_PATH = "chroma"
DATA_PATH = "data"




In [3]:
def load_documents():
    document_loader = PyPDFDirectoryLoader(DATA_PATH)
    return document_loader.load()

In [4]:
def split_text(documents: list[Document]):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=300,
        chunk_overlap=100,
        length_function=len,
        add_start_index=True,
    )
    chunks = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(chunks)} chunks.")

    document = chunks[10]
    print(document.page_content)
    print(document.metadata)

    return chunks

In [5]:
# Konfiguration des Ollama-Embeddings-Modells
ollama_emb = OllamaEmbeddings(
    model="llama3.1:8b",  # Modellname, wie auf dem Server konfiguriert
    base_url="http://192.168.180.131:11434"  # IP-Adresse und Port deines Servers
)
def save_to_chroma(chunks: list[Document]):
    # Datenbank löschen, falls sie existiert
    if os.path.exists(CHROMA_PATH):
        shutil.rmtree(CHROMA_PATH)

    # Neue Datenbank aus den Dokumenten erstellen und Ollama-Embeddings verwenden
    db = Chroma.from_documents(
        chunks, ollama_emb, persist_directory=CHROMA_PATH
    )
    db.persist()
    print(f"Saved {len(chunks)} chunks to {CHROMA_PATH}.")

In [6]:
def main():
    print("Hello World")
    generate_data_store()

    

def generate_data_store():
    documents = load_documents()
    chunks = split_text(documents)
    save_to_chroma(chunks)
    
if __name__ == "__main__":
    main()

Hello World
Split 2 documents into 28 chunks.
Sperrmüllabholung
Bis zu 10 m3 holt die FES kostenlos bei Ihnen ab.* Einen Termin erhalten 
Sie am Servicetelefon 0800 2008007-10 oder unter www.fes-frankfurt.de. In manchen Wohnsiedlungen wird Sperrmüll automatisch alle 4 Wochen abgeholt – die Termine ﬁ  nden Sie am schwarzen Brett in Ihrem Haus.
{'source': 'data/was_kommt_wo_hinein_de.pdf', 'page': 0, 'start_index': 2123}
Saved 28 chunks to chroma.


/Users/jan/Documents/GitHub/predictiv_chatbot/predictiv_chatbot/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(
